# 한국어 Tokenizing

* Tokenizing의 목적
  1. 의미를 지닌 단위로 자연어를 분절
  2. Model의 학습 시, 동일한 size로 입력
    * model에 입력시 수학적으로 (vocab 번호)입력되어야 함
    * model은 size(batch size)가 고정이 되어있기 때문에 input도 model size에 맞춰줘야함
      * batch : model의 input을 행렬로 처리
    * matrix의 끝부분을 padding 처리를 통해서 size를 맞춤
* tokenizer는 특정 사이즈로 token의 개수를 조절하는 함수가 필수로 포함되어야 함

* 한국어 tokenizing 단계
  1. 어절 단위
  2. 형태소 단위
  3. 음절 단위
  4. 자소 단위
  5. WordPiece 단위

## 실습용 데이터 준비
* '2_자연어전처리' 과정을 통해 전처리가 완료된 파일 준비

* 데이터 확인
  * `open(파일주소, 형식, encoding=)`으로 파일 읽어 옴
    * `r` : read, 파일 내용을 읽어옴
    * `encoding=` : encoding format 지정
      * 한글은 encoding format이 맞지 않으면 글자가 깨지는 현상이 발생함

In [ ]:
data = open('my_data/wiki_small.txt', 'r', encoding='utf-8')

In [ ]:
lines = data.readlines() # 전체 문장을 list에 저장하는 함수입니다.

In [ ]:
for line in lines[0:10]:
    print(line)

## 1. 어절 단위 tokenizing

* 어절 단위의 tokenizing
  * 띄어쓰기 단위로 분리함

In [ ]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = text.split(" ")    # split 함수는 입력 string에 대해서 특정 string을 기반으로 분리해줍니다.
print(tokenized_text)  

* padding 처리
  * vocab에 'padding'에 대한 번호가 존재한다면 tokenize_text에 자연어로 들어가도 됨

* 아래 예제에서 token이 4개이고 `max_seq_length`가 10이기 때문에 'padding'이 6개가 추가됨
  * 이 기능이 BERT에 내장되어 있음

In [ ]:
# 예제
max_seq_length = 10
# padding
tokenized_text += ["padding"] * (max_seq_length - len(tokenized_text))
print(tokenized_text)

* 아래 예제에서 token이 4개이고 `max_seq_length`가 2이기 때문에 앞에서부터 2개의 token을 제외한 나머지를 잘라냄(truncation)
  * 이 기능이 BERT에 내장되어 있음

In [ ]:
# 예제
max_seq_length = 2
# filtering
tokenized_text = tokenized_text[0:max_seq_length]
print(tokenized_text)

* tokenizer class 구현
  * token의 개수가 부족할 때 padding 처리하고, 개수가 많을 때는 token을 잘라서 반환하는 함수 구현

In [ ]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [ ]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "word"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "word"))

## 2. 형태소 단위 tokenizing

* 형태소 분석기 Mecab 사용
  '2_자연어처리'에서 사용되었음

In [ ]:
# !pip install konlpy

In [ ]:
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from konlpy.tag import Mecab

* 형태소 분석기 성능 테스트시 2개의 문장이 많이 사용됨
  * '아버지가방에들어가신다.'
  * '이순신은 조선 중기의 무신이다.'
    * 이순신 -> PS
    * 조선 -> DT TI
    * 중기 -> TI
    * 무신 -> OC
    * 이순신 - 직업 - 무신
    * 이순신 - 출생지 - 조선

In [ ]:
# 예제
print(mecab.pos("아버지가방에들어가신다."))

In [ ]:
mecab = Mecab()
text = "이순신은 조선 중기의 무신이다."

tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
print(tokenized_text)

* `Tokenizer` class에 형태소 tokenizer를 추가

In [ ]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [ ]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "morph"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "morph"))

## 3. 음절 단위 tokenizing

* 하나의 자연어를 한 글자씩 분리함

* `list()`로 음절단위 tokenizing함

In [ ]:
# 예제
text = "이순신은 조선 중기의 무신이다."
tokenized_text = list(text)    # split 함수는 입력 string에 대해서 특정 string을 기반으로 분리해줍니다.
print(tokenized_text)  

* `Tokenizer` class에 음절 tokenizer 추가

In [ ]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [1]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

NameError: ignored

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "syllable"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "syllable"))

## 4. 자소 단위 tokenizing

* 한글은 하나의 문자도 최대 초성, 중성, 종성 총 3개의 자소로 분리가 가능함

* 자소를 분리할 때 대부분 유니코드를 사용함
  * python은 쉽게 자소를 분리할 수 있는 라이브러리가 존재함

* hgtk
  * 자소 분리 라이브러리

In [ ]:
# !pip install hgtk

In [ ]:
import hgtk

* `tokenized_text`에 음절 단위를 구분해주는 'ᴥ'기호가 포함되어 있음
  * 음절 구분이 되어있지 않으면 혼란이 올 수 있음

In [ ]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = list(hgtk.text.decompose(text))
print(tokenized_text)
# ㅇ ㅣ ㅅ ㅜ ㄴ ㅅ ㅣ ... # 음절 구분이 어려움 
# ['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ', 'ㅓ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅜ', 'ㅇ', 'ᴥ', 'ㄱ', 'ㅣ', 'ᴥ', 'ㅇ', 'ㅢ', 'ᴥ', ' ', 'ㅁ', 'ㅜ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', 'ㄷ', 'ㅏ', 'ᴥ', '.']

* `Tokenizer` class에 자소 단위 tokenizer 추가

In [ ]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable", "jaso"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        elif tokenizer_type == "jaso":
            tokenized_text = list(hgtk.text.decompose(text))
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [ ]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "jaso"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "jaso"))

## 5. WordPiece tokenizing

In [ ]:
# !pip install transformers

In [ ]:
# !mkdir wordPieceTokenizer

* `BertWordPieceTokenizer`
  * BERT를 위한 word piece tokenizer
  * vocabulary dictionary를 생성함

  
* tokenizer 에 대한 옵션
  * `clean_text`: 기대하는 tokenizer의 형태
    * `True` : input sequence의 단어 사이의 띄어쓰기 제거
      * ex. [이순신, ##은, ' ', 조선] -> [이순신, ##은, 조선]
      * BERT에서는 '##'으로 어절의 위치를 구분할 수 있기 때문에 띄어쓰기가 없어도 됨
  * `handle_chinese_chars` : 중국어, 일본어 분리 방법
    * `True` : 음절 단위로 분리
      * vocab도 분리됨
      * '##'이 붙지 않음
  * `strip_accents` : camel case로 작성된 문자열을 분리여부
    * `True` : camel case로 작성된 문자열을 분리함
  * `lowercase` : 모든 알파벳을 소문자로 변경여부
    * `True` : 모든 알파벳을 소문자로 변경
    * `False`가 성능이 더 좋음

* `train()` 옵션
  * `files` : 파일명(주소포함) 입력
    * corpus를 input으로 넣음
  * `vocab_size` : 만들고 싶은 vocab의 size
    * vocab을 전부 채울때까지 wordpiece 알고리즘이 동작하기 때문에, size가 크면 음절단위로 만들어짐
  * `min_frequency` : 단어 등장 최소 빈도 수
    * `min_frequency=2` : 2번 이상 등장한 경우에만 vocab에 추가
  * `special_tokens` : 내부적으로 이미 정의되어 있음
    * "[PAD]" : padding token
    * "[UNK]" : unknown token
    * "[CLS]" : 맨 앞에 붙는 token
    * "[SEP]" : 맨 뒤에 붙는 token
    * "[MASK]" : masking을 위한 token
  * `wordpieces_prefix="##"` : prefix를 '##'으로 지정

* `save_model(directory, )`
  * model을 저장함

In [ ]:
from tokenizers import BertWordPieceTokenizer

# Initialize an empty tokenizer
wp_tokenizer = BertWordPieceTokenizer(
    clean_text=True,    # [이순신, ##은, ' ', 조선]
    handle_chinese_chars=True,
    strip_accents=False,    # True: [YepHamza] -> [Yep, Hamza]
    lowercase=False,
)

# And then train
wp_tokenizer.train(
    files="my_data/wiki_20190620_small.txt",
    vocab_size=10000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=1000,
    wordpieces_prefix="##"
)

# Save the files
wp_tokenizer.save_model("wordPieceTokenizer", "my_tokenizer")

In [ ]:
print(wp_tokenizer.get_vocab_size())

* tokenizer의 `encode()`함수를 통해 문장을 encoding할 수 있음

* `print(tokenized_text)`
  * transformers에서 정의한 `Encoding` class 내에 token의 개수(num_tokens), 가져올 수 있는 정보(attributes)
    * ids : vocab id
    * type_ids : BERT에서는 segment type

In [ ]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = wp_tokenizer.encode(text)
print(tokenized_text)
# Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
print(tokenized_text.tokens)
# ['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.']
print(tokenized_text.ids)
# [706, 1246, 7604, 2000, 754, 2602, 453, 8470, 1031, 16]

* `Tokenizer` class에 wordpiece tokenizer 추가

In [ ]:
class Tokenizer:
    def __init__(self):
        self.tokenizer_type_list = ["word", "morph", "syllable", "jaso", "wordPiece"]
        self.pad_token = "<pad>"
        self.max_seq_length = 10
        self.padding = False
    def tokenize(self, text, tokenizer_type): 
        assert tokenizer_type in self.tokenizer_type_list, "정의되지 않은 tokenizer_type입니다."
        if tokenizer_type == "word":
            tokenized_text = text.split(" ")
        elif tokenizer_type == "morph":
            tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
        elif tokenizer_type == "syllable":
            tokenized_text = list(text)
        elif tokenizer_type == "jaso":
            tokenized_text = list(hgtk.text.decompose(text))
        elif tokenizer_type == "wordPiece":
            tokenized_text = wp_tokenizer.encode(text).tokens
        if self.padding:
            tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
            return tokenized_text[:self.max_seq_length]
        else:
            return tokenized_text[:self.max_seq_length]
    def batch_tokenize(self, texts, tokenizer_type):
        for i, text in enumerate(texts):
            texts[i] = self.tokenize(text, tokenizer_type)
        return texts

In [ ]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = "[PAD]"
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "wordPiece"))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], "wordPiece"))

* 구현된 tokenizing 함수 확인

In [ ]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "word"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "morph"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "syllable"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "jaso"))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다.", "wordPiece"))